In [2]:
from datetime import date

class LoyaltyProgram:
    def __init__(self, program_id: str):
        self.__program_id = program_id
        self.__points = 0
        self.__rewards = []

    def add_points(self, points: int):
        assert points > 0, "Points must be positive"
        self.__points += points

    def redeem_points(self, points: int):
        if points > self.__points:
            raise ValueError("Not enough points to redeem.")
        self.__points -= points

    def __str__(self):
        return f"LoyaltyProgram(ID: {self.__program_id}, Points: {self.__points})"


class Guest:
    def __init__(self, name: str, email: str, phone: str):
        assert name.isalpha(), "Name must contain only letters"
        self.__name = name
        self.__email = email
        self.__phone = phone
        self.__loyalty_program = LoyaltyProgram(program_id=f"LP-{name[:3].upper()}")
        self.__bookings = []

    def create_account(self):
        return f"Account created for {self.__name}"

    def update_profile(self, name: str = None, phone: str = None):
        if name:
            assert name.isalpha(), "Name must contain only letters"
            self.__name = name
        if phone:
            self.__phone = phone

    def view_booking_history(self):
        return self.__bookings

    def add_booking(self, booking):
        self.__bookings.append(booking)

    def __str__(self):
        return f"Guest(Name: {self.__name}, Email: {self.__email})"


class Room:
    def __init__(self, room_number: str, room_type: str, price_per_night: float, amenities: list):
        self.__room_number = room_number
        self.__type = room_type
        self.__price = price_per_night
        self.__is_available = True
        self.__amenities = amenities

    def check_availability(self):
        return self.__is_available

    def reserve(self):
        if not self.__is_available:
            raise Exception("Room not available")
        self.__is_available = False

    def release(self):
        self.__is_available = True

    def __str__(self):
        return f"Room {self.__room_number} ({self.__type}) - ${self.__price}/night"


class Invoice:
    def __init__(self, invoice_id: str, payment_method: str):
        self.__invoice_id = invoice_id
        self.__total = 0.0
        self.__discounts = 0.0
        self.__payment_method = payment_method

    def add_charges(self, charge: float):
        assert charge >= 0, "Charge must be non-negative"
        self.__total += charge

    def apply_discount(self, discount: float):
        if discount > self.__total:
            raise ValueError("Discount cannot exceed total")
        self.__discounts += discount

    def generate_invoice(self):
        return self.__total - self.__discounts

    def __str__(self):
        return f"Invoice {self.__invoice_id}: ${self.generate_invoice():.2f} ({self.__payment_method})"


class RoomServiceRequest:
    def __init__(self, request_id: str, service_type: str):
        self.__request_id = request_id
        self.__service_type = service_type
        self.__status = "Pending"

    def submit_request(self):
        self.__status = "Submitted"

    def __str__(self):
        return f"Service Request {self.__request_id} - {self.__service_type} ({self.__status})"


class Feedback:
    def __init__(self, rating: int, comments: str):
        assert 1 <= rating <= 5, "Rating must be between 1 and 5"
        self.__rating = rating
        self.__comments = comments
        self.__date = date.today()

    def submit_feedback(self):
        return f"Feedback submitted with rating {self.__rating} on {self.__date}"

    def __str__(self):
        return f"Feedback: {self.__rating} stars - {self.__comments}"


class Booking:
    def __init__(self, booking_id: str, guest: Guest, room: Room):
        self.__booking_id = booking_id
        self.__check_in = date.today()
        self.__check_out = None
        self.__status = "Pending"
        self.__guest = guest
        self.__room = room
        self.__invoice = Invoice(f"INV-{booking_id}", payment_method="Credit Card")
        self.__services = []
        self.__feedback = None

    def confirm_booking(self):
        self.__room.reserve()
        self.__status = "Confirmed"
        self.__invoice.add_charges(self.__room._Room__price)
        self.__guest.add_booking(self)

    def cancel_booking(self):
        self.__room.release()
        self.__status = "Cancelled"

    def add_service_request(self, service: RoomServiceRequest):
        self.__services.append(service)

    def add_feedback(self, feedback: Feedback):
        self.__feedback = feedback

    def __str__(self):
        return f"Booking {self.__booking_id} - {self.__status} ({self.__room})"


In [11]:
from datetime import date, timedelta

# --------------------------
# Guest Class
# --------------------------
class Guest:
    def __init__(self, name, email, phone):
        if not name or not email or not phone:
            raise ValueError("Invalid guest details")
        self.__name = name
        self.__email = email
        self.__phone = phone

    def get_name(self):
        return self.__name

    def get_email(self):
        return self.__email


# --------------------------
# Room Class
# --------------------------
class Room:
    def __init__(self, room_number, room_type, price_per_night, amenities):
        if not room_number or price_per_night < 0:
            raise ValueError("Invalid room data")
        self.__room_number = room_number
        self.__room_type = room_type
        self.__price_per_night = price_per_night
        self.__amenities = amenities
        self.__is_available = True

    def get_price(self):
        return self.__price_per_night

    def is_available(self):
        return self.__is_available

    def reserve(self):
        self.__is_available = False

    def release(self):
        self.__is_available = True

    def get_room_number(self):
        return self.__room_number


# --------------------------
# Invoice Class
# --------------------------
class Invoice:
    def __init__(self, booking):
        self.__booking = booking
        self.__payment_method = None

    def calculate_total(self):
        days = (self.__booking["check_out"] - self.__booking["check_in"]).days
        return days * self.__booking["room"].get_price()

    def pay(self, method):
        self.__payment_method = method

    def get_payment_method(self):
        return self.__payment_method


# --------------------------
# RoomServiceRequest Class
# --------------------------
class RoomServiceRequest:
    def __init__(self, request_id, booking_id, service_type):
        self.__id = request_id
        self.__booking = booking_id
        self.__type = service_type
        self.__status = "Pending"

    def mark_complete(self):
        self.__status = "Completed"

    def get_status(self):
        return self.__status


# --------------------------
# Feedback Class
# --------------------------
class Feedback:
    def __init__(self, booking_id, rating, comment):
        self.__booking = booking_id
        self.__rating = rating
        self.__comment = comment

    def get_comment(self):
        return self.__comment

    def get_rating(self):
        return self.__rating


# --------------------------
# Test Case 1: Guest Creation
# --------------------------
guest1 = Guest("Ahmed Mohamed", "ahmed@gmail.com", "0501234567")
guest2 = Guest("Abdulla Hamdan", "abdulla@gmail.com", "0507654321")
print("Test Case 1 - Guest Info:")
print("Name:", guest1.get_name(), "| Email:", guest1.get_email())
print("Name:", guest2.get_name(), "| Email:", guest2.get_email())

# --------------------------
# Test Case 2: Room Search
# --------------------------
room1 = Room("101", "Single", 100.0, ["Wi-Fi"])
room2 = Room("102", "Suite", 250.0, ["Wi-Fi", "Jacuzzi"])
available_rooms = [r for r in [room1, room2] if r.is_available() and r.get_price() <= 200]
print("\nTest Case 2 - Available Rooms (<= $200):")
for room in available_rooms:
    print("Room:", room.get_room_number(), "| Price:", room.get_price())

# --------------------------
# Test Case 3: Make Reservation
# --------------------------
check_in = date.today()
check_out = check_in + timedelta(days=2)

booking1 = {
    "id": "B001",
    "guest": guest1,
    "room": room1,
    "check_in": check_in,
    "check_out": check_out
}
room1.reserve()

booking2 = {
    "id": "B002",
    "guest": guest2,
    "room": room2,
    "check_in": check_in,
    "check_out": check_out
}
room2.reserve()

print("\nTest Case 3 - Rooms Reserved:")
print("Guest:", booking1["guest"].get_name(), "| Room:", booking1["room"].get_room_number())
print("Guest:", booking2["guest"].get_name(), "| Room:", booking2["room"].get_room_number())

# --------------------------
# Test Case 4: Booking Confirmation
# --------------------------
print("\nTest Case 4 - Booking Confirmation:")
print(f"Booking {booking1['id']} confirmed for {booking1['guest'].get_name()}.")
print(f"Booking {booking2['id']} confirmed for {booking2['guest'].get_name()}.")

# --------------------------
# Test Case 5: Invoice Generation
# --------------------------
invoice1 = Invoice(booking1)
invoice2 = Invoice(booking2)
print("\nTest Case 5 - Invoice Total:")
print("Ahmed's total:", invoice1.calculate_total())
print("Abdulla's total:", invoice2.calculate_total())

# --------------------------
# Test Case 6: Payment Method
# --------------------------
invoice1.pay("Credit Card")
invoice2.pay("Mobile Wallet")
print("\nTest Case 6 - Payment Info:")
print("Ahmed Payment Method:", invoice1.get_payment_method())
print("Abdulla Payment Method:", invoice2.get_payment_method())

# --------------------------
# Test Case 7: Reservation History
# --------------------------
print("\nTest Case 7 - Reservation History:")
print(guest1.get_name(), "has booking ID:", booking1["id"])
print(guest2.get_name(), "has booking ID:", booking2["id"])

# --------------------------
# Test Case 8: Cancel Reservation
# --------------------------
room1.release()
room2.release()
print("\nTest Case 8 - Room Cancellation:")
print("Room", room1.get_room_number(), "Availability:", room1.is_available())
print("Room", room2.get_room_number(), "Availability:", room2.is_available())

# --------------------------
# Test Case 9: Room Service Request
# --------------------------
service1 = RoomServiceRequest("SR001", booking1["id"], "Housekeeping")
service2 = RoomServiceRequest("SR002", booking2["id"], "Laundry")
service1.mark_complete()
service2.mark_complete()
print("\nTest Case 9 - Room Service:")
print("Ahmed's service status:", service1.get_status())
print("Abdulla's service status:", service2.get_status())

# --------------------------
# Test Case 10: Feedback
# --------------------------
review1 = Feedback("B001", 5, "Very clean and fast service")
review2 = Feedback("B002", 4, "Nice room, but internet was slow")
print("\nTest Case 10 - Guest Feedback:")
print("Ahmed Rating:", review1.get_rating(), "| Comment:", review1.get_comment())
print("Abdulla Rating:", review2.get_rating(), "| Comment:", review2.get_comment())


Test Case 1 - Guest Info:
Name: Ahmed Mohamed | Email: ahmed@gmail.com
Name: Abdulla Hamdan | Email: abdulla@gmail.com

Test Case 2 - Available Rooms (<= $200):
Room: 101 | Price: 100.0

Test Case 3 - Rooms Reserved:
Guest: Ahmed Mohamed | Room: 101
Guest: Abdulla Hamdan | Room: 102

Test Case 4 - Booking Confirmation:
Booking B001 confirmed for Ahmed Mohamed.
Booking B002 confirmed for Abdulla Hamdan.

Test Case 5 - Invoice Total:
Ahmed's total: 200.0
Abdulla's total: 500.0

Test Case 6 - Payment Info:
Ahmed Payment Method: Credit Card
Abdulla Payment Method: Mobile Wallet

Test Case 7 - Reservation History:
Ahmed Mohamed has booking ID: B001
Abdulla Hamdan has booking ID: B002

Test Case 8 - Room Cancellation:
Room 101 Availability: True
Room 102 Availability: True

Test Case 9 - Room Service:
Ahmed's service status: Completed
Abdulla's service status: Completed

Test Case 10 - Guest Feedback:
Ahmed Rating: 5 | Comment: Very clean and fast service
Abdulla Rating: 4 | Comment: Nice ro